In [56]:
from pyspark.sql.functions import *
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from operator import add
df_original = spark.read.csv('data/flights.csv', header=True, inferSchema=True)
df=df_original.select('*')

In [57]:
#import some Python packages that we need:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.functions import udf
from pyspark.mllib.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.param import Param, Params
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.stat import Statistics
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from IPython.display import display
from ipywidgets import interact
import sys
import numpy as np
import pandas as pd
import time
import datetime

In [3]:
df.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (null

# 1. cleaning
## 1.1 times and dates

In [2]:
df=df.drop("YEAR")
df.printSchema()

root
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (nullable = true)
 |-- DIVERTED: integer (

In [23]:
months = [31,28,31,30,31,30,31,31,30,31,30,31]
accu=0
accu_months=[0]
for element in months:
    accu +=element
    accu_months.append(accu)
print accu_months

[0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334, 365]


In [41]:
# add column "NEW_DAY"
transform = udf(lambda month:accu_months[month-1],IntegerType())
df=df.select('*',(transform(df['MONTH'])+df['DAY']).alias('NEW_DAY'))


+-------+
|NEW_DAY|
+-------+
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
+-------+
only showing top 20 rows



In [42]:
# drop column MONTH and DAY
df=df.drop('MONTH')
df=df.drop('DAY')

root
 |-- YEAR: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (nullable = true)
 |-- DIVERTED: integer (nullable = true)
 |-- CANCELLED: inte

## 2 create labels for classification task

In [18]:
def transform_label_class_func(minutes):
    if minutes<30:
        return 1
    if minutes>=30 and minutes <=120:
        return 2
    if minutes>120:
        return 3

transform_label_class = udf(lambda minutes:transform_label_class_func(minutes),IntegerType())
temp=df.select((transform_label_class(df['DEPARTURE_DELAY'])).alias('label_class'))


count the frequency of each class

In [24]:
df.groupBy('AIRLINE').count().show()

+-------+-------+
|AIRLINE|  count|
+-------+-------+
|     UA| 515723|
|     NK| 117379|
|     AA| 725984|
|     EV| 571977|
|     B6| 267048|
|     DL| 875881|
|     OO| 588353|
|     F9|  90836|
|     US| 198715|
|     MQ| 294632|
|     HA|  76272|
|     AS| 172521|
|     VX|  61903|
|     WN|1261855|
+-------+-------+



In [34]:
df.groupBy('DESTINATION_AIRPORT').count().show()

+-------------------+-----+
|DESTINATION_AIRPORT|count|
+-------------------+-----+
|                BGM|  264|
|                INL|  574|
|                PSE|  751|
|                DLG|   77|
|              12888|   53|
|                MSY|38802|
|                PPG|  107|
|              12003|  190|
|              15041|   62|
|                GEG| 9504|
|                SNA|37195|
|                BUR|18890|
|                GRB| 4883|
|                GTF| 1966|
|              14986|  306|
|              13851| 1639|
|                IDA| 2247|
|              15412|  629|
|              11150|  112|
|                GRR|10840|
+-------------------+-----+
only showing top 20 rows



In [36]:
df_temp=df.select('*')

In [47]:
df_temp.groupBy('ORIGIN_AIRPORT').count().count().show()

628

In [38]:
df_model=df_temp
stringIndexer1 = StringIndexer(inputCol="ORIGIN_AIRPORT", outputCol="originIndex")
model_stringIndexer = stringIndexer1.fit(df_model)
indexedOrigin = model_stringIndexer.transform(df_model)
encoder1 = OneHotEncoder(dropLast=False, inputCol="originIndex", outputCol="originVec")
df_model = encoder1.transform(indexedOrigin)
df_model.groupBy('originVec').count().show()

+-----------------+-----+
|        originVec|count|
+-----------------+-----+
| (628,[32],[1.0])|43172|
| (628,[26],[1.0])|63906|
|(628,[445],[1.0])|  282|
|(628,[541],[1.0])|   88|
| (628,[81],[1.0])|11058|
| (628,[51],[1.0])|24118|
|(628,[282],[1.0])| 1281|
|(628,[336],[1.0])|  721|
|(628,[542],[1.0])|   88|
|(628,[474],[1.0])|  221|
|(628,[146],[1.0])| 4746|
|(628,[411],[1.0])|  460|
|(628,[396],[1.0])|  532|
|(628,[503],[1.0])|  159|
|(628,[421],[1.0])|  399|
|(628,[527],[1.0])|  116|
|(628,[272],[1.0])| 1420|
|(628,[406],[1.0])|  491|
|(628,[549],[1.0])|   87|
|(628,[137],[1.0])| 5261|
+-----------------+-----+
only showing top 20 rows



In [58]:
df_model=df_original.select('*')
df_model=df_model.na.drop(how='any')
stringIndexer1 = StringIndexer(inputCol='ORIGIN_AIRPORT', outputCol='originIndex')
model_stringIndexer = stringIndexer1.fit(df_model)
indexedOrigin = model_stringIndexer.transform(df_model)
encoder1 = OneHotEncoder(dropLast=False, inputCol="originIndex", outputCol="originVec")
df_model = encoder1.transform(indexedOrigin)

assembler = VectorAssembler(
    inputCols = ['MONTH','DAY','DAY_OF_WEEK','DEPARTURE_TIME','DISTANCE','originVec'],
    outputCol = 'features')
output = assembler.transform(df_model)
#airlineRDD=output.map(lambda row: LabeledPoint([0,1][row['DEPARTURE_DELAY']],row['features']))

Traceback (most recent call last):
  File "/tmp/kernel-PySpark-f6808e9c-cccc-4a3f-b66c-518bac8e97d3/pyspark_runner.py", line 196, in <module>
    output.reset()
  File "/home/andy/spark-2.2.0-bin-hadoop2.7/python/pyspark/sql/dataframe.py", line 1020, in __getattr__
    "'%s' object has no attribute '%s'" % (self.__class__.__name__, name))
AttributeError: 'DataFrame' object has no attribute 'reset'
